In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install tsfresh

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
"""
reducing.py
Author: Kirgsn, 2018
https://www.kaggle.com/code/etremblay/fail-safe-parallel-memory-reduction/notebook?scriptVersionId=27504964
https://wkirgsn.github.io/2018/02/10/auto-downsizing-dtypes/
[alternative] https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
              (tener en cuenta) https://www.kaggle.com/c/champs-scalar-coupling/discussion/96655

Use like this:
>>> from reducing import Reducer
>>> df = Reducer().reduce(df)
"""
import numpy as np
import pandas as pd
import time
import gc
# from joblib import Parallel, delayed
# from tqdm import tqdm

__all__ = ["Reducer"]


class Reducer:
    """
    Class that takes a dict of increasingly big numpy datatypes to transform
    the data of a pandas dataframe into, in order to save memory usage.
    """

    memory_scale_factor = 1024**2  # memory in MB

    def __init__(self, conv_table=None, use_categoricals=True, n_jobs=-1):
        """
        :param conv_table: dict with np.dtypes-strings as keys
        :param use_categoricals: Whether the new pandas dtype "Categoricals"
                shall be used
        :param n_jobs: Parallelization rate
        """

        self.conversion_table = conv_table or {
            "int": [np.int8, np.int16, np.int32, np.int64],
            "uint": [np.uint8, np.uint16, np.uint32, np.uint64],
            "float": [np.float32],
            "datetime": []
        }
        self.null_int = [
            pd.Int8Dtype,
            pd.Int16Dtype,
            pd.Int32Dtype,
            pd.Int64Dtype,
            pd.UInt8Dtype,
            pd.UInt16Dtype,
            pd.UInt32Dtype,
            pd.UInt64Dtype,
        ]

        self.use_categoricals = use_categoricals
        self.n_jobs = n_jobs

    def _type_candidates(self, k):
        for c in self.conversion_table[k]:
            i = np.iinfo(c) if "int" in k else np.finfo(c)
            yield c, i


    def reduce(self, df, verbose=False):
        """Takes a dataframe and returns it with all data transformed to the
        smallest necessary types.

        :param df: pandas dataframe
        :param verbose: If True, outputs more information
        :return: pandas dataframe with reduced data types
        """
        mem_usage_orig = df.memory_usage().sum() / self.memory_scale_factor
        start_time = time.time()

        gc.enable()
        df_reduced = df.apply(self._reduce, axis=0, args=(verbose,), n_jobs=self.n_jobs)

        del df
        gc.collect()

        mem_usage_new = df_reduced.memory_usage().sum() / self.memory_scale_factor
        end_time = time.time()
        print(
            f"Reduced file from {mem_usage_orig:.2f} MB "
            f"to {mem_usage_new:.2f} MB "
            f"in {(end_time - start_time):.2f} seconds"
        )

        return df_reduced

    def _reduce(self, s, verbose, n_jobs):
        try:
            isnull = False
            # skip NaNs
            if s.isnull().any():
                isnull = True
            # detect kind of type
            coltype = s.dtype
            if np.issubdtype(coltype, np.integer):
                conv_key = "int" if s.min() < 0 else "uint"
            elif np.issubdtype(coltype, np.floating):
                conv_key = "float"
            else:
                conv_key = coltype.name

            # get smallest type
            if conv_key in self.conversion_table:
                candidates = self._type_candidates(conv_key)
                for i, (t, info) in enumerate(candidates):
                    if s.max() <= info.max and s.min() >= info.min:
                        if verbose:
                            print(f"{s.name}: {coltype} -> {t}")
                        if isnull:
                            s = s.astype(t, copy=False)
                            s = s.where(s.notnull(), None)
                        else:
                            s = s.astype(t, copy=False)
                        break
            elif self.use_categoricals and "category" not in conv_key:
                if verbose:
                    print(f"{s.name}: {coltype} -> category")
                s = s.astype("category")
            return s
        except Exception:
            print(f"Error in column {s.name}: {sys.exc_info()[0]}")
            return s

In [ ]:
file_path = '/content/drive/MyDrive/dmeyf/competencia_02.csv.gz'

In [ ]:
# Definir el tamaño del bloque para procesar en cada iteración
block_size = 10000

# Inicializar un DataFrame vacío para almacenar los resultados undersampleados
undersampled_data = pd.DataFrame()

# Leer el archivo CSV en bloques
for chunk in pd.read_csv(file_path, chunksize=block_size):

    # Separar los bloques por clases
    clase_baja_1 = chunk[chunk['clase_ternaria'] == 'BAJA+1']
    clase_baja_2 = chunk[chunk['clase_ternaria'] == 'BAJA+2']
    clase_continua = chunk[chunk['clase_ternaria'] == 'CONTINUA']

    undersampled_continua = resample(clase_continua, replace=False, n_samples=max(10, int((len(clase_baja_1)+len(clase_baja_2))/2)), random_state=42)

    # Concatenar los bloques undersampleados al DataFrame final
    undersampled_data = pd.concat([undersampled_data, clase_baja_1, clase_baja_2, undersampled_continua])


In [ ]:
df = undersampled_data.copy()

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/dmeyf/competencia_02.csv.gz')
df = Reducer().reduce(df)

Reduced file from 67.82 MB to 24.34 MB in 0.39 seconds


In [ ]:
df

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
186,29293076,201901,1,0,1,50,296,1315.869995,13185.629883,1197.930054,...,22.0,5354.009766,-234.600006,0.00,7464.0,949.500000,5.0,0.0,480.929993,BAJA+1
263,29336962,201901,0,0,0,49,22,647.599976,-345.230011,649.359985,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAJA+1
492,29484324,201901,1,0,1,61,133,82.559998,10021.370117,98.570000,...,8.0,226.770004,-249.449997,0.00,3223.0,222.869995,1.0,0.0,11.730000,BAJA+1
778,29640968,201901,1,0,1,74,237,2164.169922,6524.290039,1368.869995,...,1.0,0.000000,NaN,NaN,668.0,NaN,NaN,NaN,0.000000,BAJA+1
899,29698654,201901,1,0,0,64,149,972.049988,9668.259766,725.869995,...,22.0,4363.560059,-6240.359863,0.00,4531.0,4692.109863,3.0,0.0,258.059998,BAJA+1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4562271,185397188,202107,1,0,0,28,6,7958.720215,27994.939453,2.810000,...,10.0,0.000000,NaN,NaN,158.0,NaN,NaN,NaN,0.000000,CONTINUA
4560565,178583188,202107,1,0,0,30,14,633.229980,5112.899902,1120.660034,...,3.0,0.000000,-8310.929688,0.00,418.0,0.000000,0.0,0.0,703.799988,CONTINUA
4561700,182231149,202107,1,0,0,32,11,129.389999,9718.450195,112.919998,...,3.0,0.000000,-20286.910156,0.00,332.0,8541.500000,3.0,0.0,1489.709961,CONTINUA
4562230,185317401,202107,1,0,0,44,6,3129.030029,10378.589844,2304.729980,...,3.0,0.000000,-17434.330078,9.46,179.0,2056.100098,3.0,0.0,2275.620117,CONTINUA


In [ ]:
diccionario_datos  = pd.read_excel('/content/drive/MyDrive/dmeyf/DiccionarioDatos_2023.xlsx')

In [ ]:
columnas_pesos = diccionario_datos[diccionario_datos['unidad'] == 'pesos']['campo'].tolist()

In [ ]:
df.columns

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'Visa_fultimo_cierre', 'Visa_mpagado', 'Visa_mpagospesos',
       'Visa_mpagosdolares', 'Visa_fechaalta', 'Visa_mconsumototal',
       'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo',
       'clase_ternaria'],
      dtype='object', length=155)

In [ ]:
# Crear nuevas columnas divididas por el promedio para cada foto_mes
no_encontrada = []
for columna in columnas_pesos:
    # columna = columna.lower()
    if columna in df.columns:
        promedio_por_mes = df.groupby('foto_mes')[columna].transform('mean')
        nueva_columna = f'{columna}_div_promedio'
        df[nueva_columna] = df[columna] / promedio_por_mes
        df = df.drop(columna, axis=1)
    else:
        no_encontrada.append(columna)

print(no_encontrada)

['mtarjeta_visa_debitos_automaticos']


In [ ]:
len(no_encontrada)

1

In [ ]:
df

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,cproductos,tcuentas,ccuenta_corriente,...,Visa_mconsumospesos_div_promedio,Visa_mconsumosdolares_div_promedio,Visa_mlimitecompra_div_promedio,Visa_madelantopesos_div_promedio,Visa_madelantodolares_div_promedio,Visa_mpagado_div_promedio,Visa_mpagospesos_div_promedio,Visa_mpagosdolares_div_promedio,Visa_mconsumototal_div_promedio,Visa_mpagominimo_div_promedio
186,29293076,201901,1,0,1,50,296,7,1,1,...,0.111323,0.000000,0.324952,0.0,0.0,3.301436,0.019783,0.000000,0.111323,0.026627
263,29336962,201901,0,0,0,49,22,4,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,29484324,201901,1,0,1,61,133,9,1,1,...,0.026130,0.000000,1.772344,0.0,0.0,0.139833,0.021036,0.000000,0.026130,0.000649
778,29640968,201901,1,0,1,74,237,6,1,1,...,NaN,NaN,0.541587,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000
899,29698654,201901,1,0,0,64,149,7,1,1,...,0.550121,0.000000,0.433270,0.0,0.0,2.690696,0.526240,0.000000,0.550121,0.014288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4562271,185397188,202107,1,0,0,28,6,8,1,1,...,NaN,NaN,1.365380,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000
4560565,178583188,202107,1,0,0,30,14,7,1,1,...,0.000000,0.000000,0.507020,NaN,NaN,0.000000,0.182592,0.000000,0.000000,0.109582
4561700,182231149,202107,1,0,0,32,11,7,1,1,...,0.408309,0.000000,0.190132,NaN,NaN,0.000000,0.445706,0.000000,0.408309,0.231948
4562230,185317401,202107,1,0,0,44,6,7,1,1,...,0.098288,0.562382,0.760530,NaN,NaN,0.000000,0.383034,0.431728,0.098288,0.354314


In [ ]:
# 1. Convertir la columna 'foto_mes' a formato de fecha
df['foto_mes'] = pd.to_datetime(df['foto_mes'], format='%Y%m')

# 2. Asegurarte de tener una columna 'id'
df['id'] = df['numero_de_cliente']

# 3. Identificar columnas numéricas para aplicar interpolación
columnas_numericas = df.select_dtypes(include='number').columns

# 4. Aplicar interpolación solo a columnas numéricas
df[columnas_numericas] = df[columnas_numericas].interpolate(method='linear', axis=0)
df = df.ffill().bfill()

df_numeric = df[columnas_numericas]
df_numeric['foto_mes'] = df['foto_mes']

# 6. Extraer características temporales con TSFRESH
extracted_features = extract_features(df_numeric, column_id='id', column_sort='foto_mes', n_jobs=16)

# 7. Agregar la columna 'clase_ternaria' a las características extraídas
extracted_features['clase_ternaria'] = df.groupby('id')['clase_ternaria'].first()

# 8. Imputar valores faltantes
extracted_features = impute(extracted_features)

# Ahora, `extracted_features` contiene las características temporales generadas por TSFRESH con imputación de valores NaN


<ipython-input-18-75471f30ebdf>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric['foto_mes'] = df['foto_mes']
Feature Extraction:   0%|          | 0/80 [00:00<?, ?it/s]